# Benchmark Model: Raw Baseline

## Purpose
This notebook establishes a performance baseline using a simple XGBoost model trained on only **6 raw features** with minimal preprocessing. This satisfies the assignment requirement for a "simple heuristic or a model with just a couple of features" to serve as a comparison point.

## Prerequisites
- Model registry (`flight-delay-models` group) must exist (created in model registry setup notebook)
- Feature data available in public S3 bucket: `s3://sagemaker-us-east-1-425709451100/aai540-group1/features/`

## Benchmark Strategy
We use only the most basic temporal and flight characteristics:
- `MONTH`, `DAY_OF_WEEK`, `DEP_HOUR` (temporal)
- `DISTANCE`, `SCHEDULED_TIME` (flight characteristics)
- `IS_WEEKEND` (derived temporal flag)

This model will be registered to the model group for comparison with more sophisticated approaches in subsequent notebooks.

In [1]:
import boto3
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.inputs import TrainingInput

print("Imports loaded successfully")
print(f"SageMaker version: {sagemaker.__version__}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Imports loaded successfully
SageMaker version: 2.245.0


In [2]:
# SageMaker session and role
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

# S3 buckets
PUBLIC_BUCKET = 'sagemaker-us-east-1-425709451100'
PERSONAL_BUCKET = sess.default_bucket()

# Benchmark features (6 only - minimal baseline, are required by this week's Team Project Steps)
BENCHMARK_FEATURES = [
    'MONTH', 
    'DAY_OF_WEEK', 
    'DEP_HOUR', 
    'DISTANCE', 
    'SCHEDULED_TIME', 
    'IS_WEEKEND'
]
TARGET = 'DELAYED'

# S3 paths
INPUT_PREFIX = 'aai540-group1/features'
OUTPUT_PREFIX = 'aai540-group1/training/raw-baseline'

print(f"Region: {region}")
print(f"Role: {role.split('/')[-1]}")
print(f"Public bucket (features): s3://{PUBLIC_BUCKET}/{INPUT_PREFIX}/")
print(f"Personal bucket (training): s3://{PERSONAL_BUCKET}/{OUTPUT_PREFIX}/")
print(f"\nBenchmark features ({len(BENCHMARK_FEATURES)}): {BENCHMARK_FEATURES}")
print(f"Target: {TARGET}")

Region: us-east-1
Role: LabRole
Public bucket (features): s3://sagemaker-us-east-1-425709451100/aai540-group1/features/
Personal bucket (training): s3://sagemaker-us-east-1-786869526001/aai540-group1/training/raw-baseline/

Benchmark features (6): ['MONTH', 'DAY_OF_WEEK', 'DEP_HOUR', 'DISTANCE', 'SCHEDULED_TIME', 'IS_WEEKEND']
Target: DELAYED


In [11]:
# Load feature data from public bucket
s3_client = boto3.client('s3')

print("Loading feature data from public bucket...")

datasets = {}
for split in ['train', 'val', 'test']:
    s3_path = f"s3://{PUBLIC_BUCKET}/{INPUT_PREFIX}/{split}_features.parquet"
    
    # Read only needed columns from parquet (efficient columnar read)
    # Parquet's columnar format allows reading specific columns without loading entire file
    columns_to_read = BENCHMARK_FEATURES + [TARGET]
    df_benchmark = pd.read_parquet(s3_path, columns=columns_to_read)
    
    datasets[split] = df_benchmark
    
    print(f"{split:5s}: {df_benchmark.shape[0]:>7,} rows × {df_benchmark.shape[1]} columns")

# Unpack datasets
train_df = datasets['train']
val_df = datasets['val']
test_df = datasets['test']

print(f"\nTotal samples: {sum(df.shape[0] for df in datasets.values()):,}")
print(f"\nFeatures: {BENCHMARK_FEATURES}")
print(f"\nSample from training set:")
print(train_df.head(3))

Loading feature data from public bucket...
train: 4,299,046 rows × 7 columns
val  : 482,878 rows × 7 columns
test : 462,367 rows × 7 columns

Total samples: 5,244,291

Features: ['MONTH', 'DAY_OF_WEEK', 'DEP_HOUR', 'DISTANCE', 'SCHEDULED_TIME', 'IS_WEEKEND']

Sample from training set:
   MONTH  DAY_OF_WEEK  DEP_HOUR  DISTANCE  SCHEDULED_TIME  IS_WEEKEND  DELAYED
0      1            4         0      1448           205.0           0        0
1      1            4         0      2330           280.0           0        0
2      1            4         0      2296           286.0           0        0


In [6]:
# Define XGBoost Estimator
from sagemaker.image_uris import retrieve

# Get XGBoost container image
container = retrieve('xgboost', region, version='1.5-1')

# Output path for model artifacts
output_path = f"s3://{PERSONAL_BUCKET}/aai540-group1/models/raw-baseline"

# Hyperparameters for benchmark model
hyperparameters = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 8,
    'eta': 0.1,
    'num_round': 500,
    'scale_pos_weight': 4.58,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'early_stopping_rounds': 30
}

# Create estimator
xgb_estimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=output_path,
    sagemaker_session=sess,
    hyperparameters=hyperparameters
)

print("XGBoost Estimator configured")
print(f"\nContainer: {container}")
print(f"Instance type: ml.m5.xlarge")
print(f"Output path: {output_path}")
print(f"\nHyperparameters:")
for key, value in hyperparameters.items():
    print(f"  {key:25s} = {value}")

XGBoost Estimator configured

Container: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1
Instance type: ml.m5.xlarge
Output path: s3://sagemaker-us-east-1-786869526001/aai540-group1/models/raw-baseline

Hyperparameters:
  objective                 = binary:logistic
  eval_metric               = auc
  max_depth                 = 8
  eta                       = 0.1
  num_round                 = 500
  scale_pos_weight          = 4.58
  subsample                 = 0.8
  colsample_bytree          = 0.8
  early_stopping_rounds     = 30


In [8]:
# Prepare data for SageMaker and train model
import os
from botocore.exceptions import ClientError

# Local temp directory
local_dir = '/tmp/benchmark_train'
os.makedirs(local_dir, exist_ok=True)

print("Preparing training data (CSV format: target first, no headers)...")

# S3 training data location
s3_train_prefix = f"{OUTPUT_PREFIX}"

# Save and upload each dataset
s3_inputs = {}
for split, df in [('train', train_df), ('validation', val_df)]:
    # Reorder: target first, then features
    df_train = df[[TARGET] + BENCHMARK_FEATURES].copy()
    
    # Save locally as CSV (no headers)
    local_path = f"{local_dir}/{split}.csv"
    df_train.to_csv(local_path, header=False, index=False)
    
    # Upload to S3
    s3_key = f"{s3_train_prefix}/{split}/{split}.csv"
    s3_path = f"s3://{PERSONAL_BUCKET}/{s3_key}"
    
    # Check if already exists
    try:
        s3_client.head_object(Bucket=PERSONAL_BUCKET, Key=s3_key)
        print(f"{split:12s}: Already exists at {s3_path}")
    except ClientError:
        s3_client.upload_file(local_path, PERSONAL_BUCKET, s3_key)
        print(f"{split:12s}: Uploaded to {s3_path}")
    
    s3_inputs[split] = s3_path

# Create TrainingInput channels
train_input = TrainingInput(s3_inputs['train'], content_type='text/csv')
validation_input = TrainingInput(s3_inputs['validation'], content_type='text/csv')

Preparing training data (CSV format: target first, no headers)...
train       : Already exists at s3://sagemaker-us-east-1-786869526001/aai540-group1/training/raw-baseline/train/train.csv
validation  : Already exists at s3://sagemaker-us-east-1-786869526001/aai540-group1/training/raw-baseline/validation/validation.csv


In [9]:

print(f"\nStarting training job...")
print(f"Train data: {s3_inputs['train']}")
print(f"Validation data: {s3_inputs['validation']}")

# Train the model
xgb_estimator.fit({
    'train': train_input,
    'validation': validation_input
}, wait=True)

print("\nTraining complete!")
print(f"Model artifacts: {xgb_estimator.model_data}")

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2026-02-02-03-34-08-692



Starting training job...
Train data: s3://sagemaker-us-east-1-786869526001/aai540-group1/training/raw-baseline/train/train.csv
Validation data: s3://sagemaker-us-east-1-786869526001/aai540-group1/training/raw-baseline/validation/validation.csv
2026-02-02 03:34:09 Starting - Starting the training job...
2026-02-02 03:34:24 Starting - Preparing the instances for training...
2026-02-02 03:35:10 Downloading - Downloading the training image......
2026-02-02 03:35:56 Training - Training image download completed. Training in progress../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2026-02-02 03:36:08.054 ip-10-2-123-238.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2026-02-02 03:36:08.076 ip-10-2-123-238.ec2.internal:7 INFO profiler_config_parser.py:111] User

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 }, wait=True)                                                                               │
│   10                                                                                             │
│   11 print("\nTraining complete!")                                                               │
│ ❱ 12 print(f"Model artifacts: {xgb_estimator.model_data}")                                       │
│   13                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py:1889 in model_data                │
│                                                                                                  │
│   1886 │   │   │   job_details = self.sagemaker_session.sagemaker_client.describe_training_job(  │
│   1887 │   │   │   │   TrainingJobName=self.latest_training_job.name                             │
│   1888 │   │   │   )                                                                             │
│ ❱ 1889 │   │   │   model_uri = job_details["ModelArtifacts"]["S3ModelArtifacts"]                 │
│   1890 │   │   │   compression_type = job_details.get("OutputDataConfig", {}).get(               │
│   1891 │   │   │   │   "CompressionType", "GZIP"                                                 │
│   1892 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'ModelArtifacts'